In [1]:
!ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
!ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF



pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 74aebb552ea7...   0% ▕                ▏    0 B/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...   0% ▕                ▏    0 B/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...   5% ▕                ▏ 3.3 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...   9% ▕█               ▏ 6.1 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...  17% ▕██              ▏  11 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...  26% ▕████            ▏  17 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...  31% ▕████            ▏  20 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...  39% ▕██████          ▏  26 MB/ 68 MB                  pulling manifest 
pulling 74aebb552ea7...  46% ▕███████         ▏  31 MB/ 68 MB            

In [2]:
pip install ollama

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.5 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.0-py3-none-any.whl (14 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Indexing Phase : convert chunk into embeddings

In [3]:
# read dataset as a list
dataset = []
with open('cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


use ollma to convert chunk into embedding vector, and store the chunk and corresponding vector in a list

In [5]:
import ollama
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'
# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []


In [6]:
def add_chunk_to_database(chunk):
    embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
    VECTOR_DB.append((chunk, embedding))

In [7]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')

Added chunk 1/150 to the database
Added chunk 2/150 to the database
Added chunk 3/150 to the database
Added chunk 4/150 to the database
Added chunk 5/150 to the database
Added chunk 6/150 to the database
Added chunk 7/150 to the database
Added chunk 8/150 to the database
Added chunk 9/150 to the database
Added chunk 10/150 to the database
Added chunk 11/150 to the database
Added chunk 12/150 to the database
Added chunk 13/150 to the database
Added chunk 14/150 to the database
Added chunk 15/150 to the database
Added chunk 16/150 to the database
Added chunk 17/150 to the database
Added chunk 18/150 to the database
Added chunk 19/150 to the database
Added chunk 20/150 to the database
Added chunk 21/150 to the database
Added chunk 22/150 to the database
Added chunk 23/150 to the database
Added chunk 24/150 to the database
Added chunk 25/150 to the database
Added chunk 26/150 to the database
Added chunk 27/150 to the database
Added chunk 28/150 to the database
Added chunk 29/150 to the dat

# 2. Retrevial Phase: Given a query and return top k chunks

In [14]:
# 2. Retrevial Phase: Given a query and return top k chunks
def cosine_similarity(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

In [18]:
def retreive(query, top_n=3):
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)["embeddings"][0]
    similarities = []
    for chunk, embedding in VECTOR_DB:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]



# 3. Generation Phrase: chatbot generate a response based on retrevived knowledge from the step above

In [23]:
input_query = input("Ask me a question!")
retrieved_knowledge = retreive(input_query) # return the top k responses 

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
    print(f' - (similarity: {similarity:.2f}) {chunk}')

context_str = '\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])
instruction_prompt = f"""
You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{context_str}"""

# what are the cats' most common personality?
# Give me a table on breed with its proportions





Retrieved knowledge:
 - (similarity: 0.67) The largest cat breed is the Ragdoll. Male Ragdolls weigh between 12 and 20 lbs. (5.4-9.0 k). Females weigh between 10 and 15 lbs. (4.5-6.8 k).

 - (similarity: 0.64) The smallest pedigreed cat is a Singapura, which can weigh just 4 lbs. (1.8 kg), or about five large cans of cat food. The largest pedigreed cats are Maine Coon cats, which can weigh 25 lbs. (11.3 kg), or nearly twice as much as an average cat weighs.

 - (similarity: 0.64) Today there are about 100 distinct breeds of the domestic cat.



In [24]:
# generate response
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
Here is a table based on the provided information:

| **Breed** | **Weight Range (lbs/kg)** |
| --- | --- |
| Ragdoll | 12-20 |
| Singapura | 4 |
| Maine Coon | 25 |

Let me know if you'd like to add more breeds!

# Put Everything Together Workflow
Ask me a question: tell me about cat speed 

Retrieved chunks: ... 

Chatbot response: 

According to the given context, cats can travel at approximately 31 mph (49 km) over a short distance. This is their top speed.


# Rooms for improvement

Prolem 1: efficiency issue, multiple topics: if have multiple topics, may not have a good answer
Solution1: 
- Let the chatbot writes its own query.
- Use multiple queries to retieve information

Problem 2: Cosine Similarity may not guarantee best similarity score
Solution 2: Use **reranking** method to rerank the retrieved chunk based on relavance to query

Problem 3: database are stored locally, maynot **scalable** for larger dataset
Solution 3: We can use a more efficient vector database such as Qdrant, Pinecone, pgvector

Problem 4: Each sentence as a chunk may not be effective
Solution 4: 
- break down the dataset into smaller chunks. 
- We can also pre-process each chunk before adding them to the database 

Problem 5: smaller llm, less capability
Solution: larger llm





# Other Types of RAG
- **Graph RAG**: In this type of RAG, the knowledge source is represented as a graph, where nodes are entities and edges are relationships between entities. The language model can traverse the graph to retrieve relevant information. There are many active researches on this type of RAG. Here is a collection of papers on Graph RAG.
- **Hybrid RAG**: a type of RAG that combines Knowledge Graphs (KGs) and vector database techniques to improve question-answering systems. To know more, you can read the paper here.
- **Modular RAG**: a type of RAG that goes beyond the basic "retrieve-then-generate" process, employing routing, scheduling, and fusion mechanisms to create a flexible and reconfigurable framework. This modular design allows for various RAG patterns (linear, conditional, branching, and looping), enabling more sophisticated and adaptable knowledge-intensive applications. To know more, you can read the paper here.